In [1]:
import age
from age.gen.ageParser import *
GRAPH_NAME = "text_test"

DSN = "host=localhost port=5432 dbname=texttest user=afidegnum password=chou1979"
ag = age.connect(graph=GRAPH_NAME, dsn=DSN)




In [9]:
import json
from lxml import etree
from itertools import tee
from lxml import html
bare = "/home/afidegnum/projects/prototypes/apps/mydemo/resources/bare.html"
dom_tree = html.parse(bare)
from bs4 import BeautifulSoup, Tag

In [11]:
import psycopg2 
import age

GRAPH_NAME = "text_test"
DSN = "host=localhost port=5432 dbname=texttest user=afidegnum password=chou1979"
conn = psycopg2.connect(DSN)

age.setUpAge(conn, GRAPH_NAME)


In [12]:
def get_nodes(tag):
    """ Get the list of nodes."""
    with conn.cursor() as cursor:
        try :
            cursor.execute("""
            SELECT * from cypher(%s, $$
            MATCH (V:node)-[connect]->(V2:node)
			      WITH V as parent, COLLECT(V2) as children
            RETURN {parent: parent, children: children}
            $$) as (map agtype);

            """, (GRAPH_NAME, tag,))
            
            rows = cursor.fetchall()
            
            r = [r[0] for r in rows]
            return(r)
                    
        except Exception as ex:
            print(ex)
            conn.rollback()
get_nodes("html")

not all arguments converted during string formatting


In [3]:
### Final
# soup = BeautifulSoup()
# cursor = ag.execCypher("MATCH (n:node {tag: 'html'}) RETURN n")    
# t = [x[0].id for x in cursor]
# print(t[0])





def get_tree(n_id):
    """ Get the trees"""
    with conn.cursor() as cursor:
        try :                    
            cursor.execute("""
            SELECT * from cypher(%s, $$
            MATCH (V:node)-[r:connect]->(V2:node)
			WITH   COLLECT(V2) as children
          RETURN  children
$$) as (children agtype);
            """, (GRAPH_NAME,))
        
            for row in cursor:
                print(row)
                            
        except Exception as ex:
            print(ex)
            conn.rollback()


node_id = 844424930131969
                                      
get_tree(node_id)


# for b in g:
#     print(b)


# file_soup = soup.prettify()
# with open("helloworld.html", "w") as file:
#     file.write(str(file_soup))

__init__() missing 1 required positional argument: 'cause'


line 1:189 mismatched input '::vertex' expecting '::edge'
line 1:408 mismatched input '::vertex' expecting '::edge'
line 1:682 mismatched input '::vertex' expecting '::edge'
line 1:910 mismatched input '::vertex' expecting '::edge'
line 1:1138 mismatched input '::vertex' expecting '::edge'
line 1:1370 mismatched input '::vertex' expecting '::edge'
line 1:1602 mismatched input '::vertex' expecting '::edge'
line 1:1832 mismatched input '::vertex' expecting '::edge'
line 1:2026 mismatched input '::vertex' expecting '::edge'
line 1:2239 mismatched input '::vertex' expecting '::edge'
line 1:2433 mismatched input '::vertex' expecting '::edge'


In [63]:
ag.execCypher("CREATE (n:node {name: 'Joe', attrib: {myclass: 'id', myid: 'this_id'}})")
ag.commit()

In [ ]:
import json
def handleElement(parentid , element):
    # TODO add support for comments. for now, we are stripping all the comments out. 
    x = {key:value for (key,value) in element.attrib.items()}
    y = json.dumps(x if x else None)
    cursor = ag.execCypher("CREATE (t:node {tag: %s, attrib: %s} ) RETURN id(t)", params=(element.tag, y,))        
    eid = cursor.fetchone()[0]  # get last inserted ID 
    print(y)
    ag.commit()
    # print("eid:", eid)

    if parentid :
        cx = ag.execCypher("MATCH (p:node),(k:node) WHERE id(p) = %s AND id(k) = %s CREATE (p)<-[:connect {name: p.name+ '<-'+k.name}]-(k)", params=(eid, parentid,))
        ag.commit()
        print("eid:", eid, " --> parent:", parentid)

    for child in element.iterchildren():
         handleElement(eid, child)

x = dom_tree.getroot()
handleElement(None, x)

SELECT * from cypher('text_test', $$
          MATCH (V:node {tag: 'html'})-[R:connect]-(V2)
          RETURN V2
$$) as (V2 agtype);

In [ ]:
def node_tree(parentid , element):
    if parentid:
       x = {f"t_{key}":value for (key,value) in element.attrib.items()}
       print(x if x else "")
    for child in element.iterchildren():
         node_tree(element, child)

x = dom_tree.getroot()
node_tree(None, x)

In [17]:
def graph_one():
    children = ag.execCypher("MATCH (V:node)-[connect]->(V2:node) WITH V as parent, COLLECT(V2) as children RETURN parent, children", cols=["a agtype", "b agtype"])
    for c in children:
        print(c)
        #graph_nodes(c[0].id)

graph_one()

({label:node, id:844424930131987, properties:{tag: div, attrib: {\"id\": \"second\", \"class\": \"below\"}, }}::VERTEX, [{label:node, id:844424930131988, properties:{tag: div, attrib: {\"class\": \"inner\"}, }}::VERTEX]::PATH)


line 1:116 missing '::path' at '<EOF>'
line 1:265 mismatched input '::vertex' expecting '::edge'
line 1:388 missing '::path' at '<EOF>'


TypeError: __init__() missing 1 required positional argument: 'cause'

In [21]:
import xml.etree.ElementTree as ET

def xml_nodes(t_id, parent=None):    
    children = ag.execCypher("MATCH (v:node)-[R:connect]->(V2) WHERE id(v) = %s RETURN V2", params=(t_id,))
    for c in children:
        if parent is None:
            parent = el = ET.Element(c[0]['tag'])
        else:
            el = ET.SubElement(parent, c[0]['tag'])
        xml_nodes(c[0].id, el)
    return parent

cursor = ag.execCypher("MATCH (n:node {tag: 'html'}) RETURN n")    
t = [x[0].id for x in cursor]
print(t[0])
xml_nodes(t[0], None)

844424930131969


<Element 'head' at 0x7f004cfa8c20>

In [28]:
soup = BeautifulSoup()

cursor = ag.execCypher("MATCH (n:node {tag: 'html'}) RETURN n")    
t = [x[0].id for x in cursor]
# print(t[0])

def html_graph(t_id):    
    parent = ag.execCypher("MATCH (V:node) WHERE id(V) = %s RETURN V", params=(t_id,))
    p = [x[0] for x in parent]
    pp = p[0]["tag"]       
    # parent_tag = soup.new_tag(name=p[0]["tag"])
    # print(f"{parent_tag}")
    #soup.append(parent_tag)
    print(pp)

    children = ag.execCypher("MATCH (v:node)-[R:connect]->(V2) WHERE id(v) = 844424930131969 RETURN v, V2", cols=['v', 'V2'], params=(p[0].id,))
    for c in children:
        print(c)

print(t[0])
html_graph(t[0])

SqlExcutionError: ("Excution ERR[function cypher(unknown, unknown) does not exist\nLINE 1: SELECT * from cypher('text_test', $$ MATCH (n:node {tag: 'ht...\n                      ^\nHINT:  No function matches the given name and argument types. You might need to add explicit type casts.\n](SELECT * from cypher('text_test', $$ MATCH (n:node {tag: 'html'}) RETURN n $$) as (v agtype);)", UndefinedFunction("function cypher(unknown, unknown) does not exist\nLINE 1: SELECT * from cypher('text_test', $$ MATCH (n:node {tag: 'ht...\n                      ^\nHINT:  No function matches the given name and argument types. You might need to add explicit type casts.\n"))

In [ ]:
soup = BeautifulSoup()

cursor = ag.execCypher("MATCH (n:node {tag: 'html'}) RETURN n")    
t = [x[0].id for x in cursor]
print(t[0])

def html_nodes(t_id):    
    parent = ag.execCypher("MATCH (n:node) WHERE id(n) = %s RETURN n", params=(t_id,))
    p = [x[0] for x in parent]
    pp = p[0]["tag"]       
    # parent_tag = soup.new_tag(name=p[0]["tag"])
    # print(f"{parent_tag}")
    soup.append(parent_tag)
    children = ag.execCypher("MATCH (v:node)-[R:connect]->(V2) WHERE id(v) = %s RETURN V2", params=(p[0].id,))
    for c in children:
        if parent is None:            
            parent_tag = el = soup.new_tag(name=c[0]["tag"])
            soup.append(parent_tag)
        else:
            el = soup.new_tag(name=c[0]["tag"])
            parent_tag.append(el)            
        html_nodes(c[0].id, el)
    return parent


html_nodes(t[0], "html")

In [2]:
def graph_ndes(t_id):
    # t = []
    children = ag.execCypher("MATCH (v:node)-[R:connect]->(V2) WHERE id(v) = %s RETURN V2", params=(t_id,))
    for c in children:
        print(c[0])
        t.append(c[0])
        graph_ndes(c[0].id)
    return(t)
    
cursor = ag.execCypher("MATCH (n:node {tag: 'html'}) RETURN n")    
t = [x[0].id for x in cursor]
# print(t[0])
dumped = graph_ndes(t[0])
print(dumped)
    

{label:node, id:844424930131970, properties:{tag: head, attrib: null, }}::VERTEX
{label:node, id:844424930131971, properties:{tag: meta, attrib: {\"charset\": \"UTF-8\"}, }}::VERTEX
{label:node, id:844424930131972, properties:{tag: title, attrib: null, }}::VERTEX
{label:node, id:844424930131973, properties:{tag: body, attrib: null, }}::VERTEX
{label:node, id:844424930131974, properties:{tag: ul, attrib: {\"class\": \"menu\"}, }}::VERTEX
{label:node, id:844424930131975, properties:{tag: div, attrib: {\"class\": \"itm\"}, }}::VERTEX
{label:node, id:844424930131976, properties:{tag: div, attrib: {\"class\": \"itm\"}, }}::VERTEX
{label:node, id:844424930131977, properties:{tag: div, attrib: {\"class\": \"itm\"}, }}::VERTEX
{label:node, id:844424930131978, properties:{tag: div, attrib: {\"id\": \"idone\", \"class\": \"classone\"}, }}::VERTEX
{label:node, id:844424930131979, properties:{tag: ul, attrib: {\"class\": \"listing\"}, }}::VERTEX
{label:node, id:844424930131980, properties:{tag: li

In [20]:
# Import Module

soup = BeautifulSoup()

cursor = ag.execCypher("MATCH (n:node {tag: 'html'}) RETURN n")    
t = [x[0].id for x in cursor]
print(t[0])

def graph_dom(t_id):
    parent = ag.execCypher("MATCH (n:node) WHERE id(n) = %s RETURN n", params=(t_id,))
    p = [x[0] for x in parent]
    pp = p[0]["tag"]       
    parent_tag = soup.new_tag(name=p[0]["tag"])
    soup.append(parent_tag)
    # d = None
    children = ag.execCypher("MATCH (v:node)-[R:connect]->(V2) WHERE id(v) = %s RETURN V2", params=(p[0].id,))
    print(f"Parent ----------{pp}")
    # print(f"Children: {children}")
    for d in children:
        check_parent = ag.execCypher("MATCH (v:node)<-[R:connect]-(V2) WHERE id(v) = %s RETURN V2", params=(d[0].id,))
        cp = [c[0] for c in check_parent]
        # print(f"{pp}---------------------")
        print(p[0].id, cp[0].id)
        if (p[0].id == cp[0].id):
            children_tag = soup.new_tag(name=d[0]["tag"])
            parent_tag.append(children_tag)
            print(d[0]["tag"])
            graph_dom(d[0].id)                         

graph_dom(t[0])

file_soup = soup.prettify()
with open("helloworld.html", "w") as file:
    file.write(str(file_soup))

844424930131969
Parent ----------html
844424930131969 844424930131969
head
Parent ----------head
844424930131970 844424930131970
meta
Parent ----------meta
844424930131970 844424930131970
title
Parent ----------title
844424930131969 844424930131969
body
Parent ----------body
844424930131973 844424930131973
ul
Parent ----------ul
844424930131974 844424930131974
div
Parent ----------div
844424930131974 844424930131974
div
Parent ----------div
844424930131974 844424930131974
div
Parent ----------div
844424930131973 844424930131973
div
Parent ----------div
844424930131978 844424930131978
ul
Parent ----------ul
844424930131979 844424930131979
li
Parent ----------li
844424930131979 844424930131979
li
Parent ----------li
844424930131979 844424930131979
li
Parent ----------li
844424930131978 844424930131978
div
Parent ----------div
844424930131983 844424930131983
h1
Parent ----------h1
844424930131978 844424930131978
div
Parent ----------div
844424930131985 844424930131985
h2
Parent ----------

In [7]:
def graphed_dom(t_id):
    parent = ag.execCypher("MATCH (n:node) WHERE id(n) = %s RETURN n", params=(t_id,))
    p = [x[0] for x in parent]
    pp = p[0]["tag"]        
    children = ag.execCypher("MATCH (v:node)-[R:connect]->(V2) WHERE id(v) = %s RETURN V2", params=(p[0].id,))
    print(f"Parent ----------{pp}")
    # print(f"Children: {children}")
    # t = []
    for d in children:
        t.append(d)
        graphed_dom(d[0].id)
    return(t)
graphed = graphed_dom(t[0])
print(graphed)

Parent ----------html
Parent ----------head
Parent ----------meta
Parent ----------title
Parent ----------body
Parent ----------ul
Parent ----------div
Parent ----------div
Parent ----------div
Parent ----------div
Parent ----------ul
Parent ----------li
Parent ----------li
Parent ----------li
Parent ----------div
Parent ----------h1
Parent ----------div
Parent ----------h2
Parent ----------div
Parent ----------div
Parent ----------h1
Parent ----------h1
Parent ----------h2
[844424930131969, {label:node, id:844424930131970, properties:{tag: head, attrib: null, }}::VERTEX, {label:node, id:844424930131971, properties:{tag: meta, attrib: {\"charset\": \"UTF-8\"}, }}::VERTEX, {label:node, id:844424930131972, properties:{tag: title, attrib: null, }}::VERTEX, {label:node, id:844424930131973, properties:{tag: body, attrib: null, }}::VERTEX, {label:node, id:844424930131974, properties:{tag: ul, attrib: {\"class\": \"menu\"}, }}::VERTEX, {label:node, id:844424930131975, properties:{tag: div, at

In [63]:
sp = BeautifulSoup()
ht = sp.new_tag("html")
th = sp.find(ht)
hd = sp.new_tag("head")
dv = soup.select_one(ht)
sp.append(dv)
bd = sp.new_tag("body")
sp.append(dv)
# sp.append(hd)
sp.append(bd)
print(sp.prettify())

TypeError: 'NoneType' object is not callable

In [4]:
ns = BeautifulSoup()
nt = ns.new_tag("html")
nb = ns.new_tag("body")
ns.append(nt)
nd = ns.new_tag("head")
nt.append(nd)
nt.append(nb)
print(nt.name)
print(ns.prettify())

html
<html>
 <head>
 </head>
 <body>
 </body>
</html>


In [5]:
f = open('helloworld.html','w')

hi = "hello, hi"

message = f"""<html>
<head></head>
<body><p>{hi}</p></body>
<div>Thist</div>
</html>"""

f.write(message)
f.close()

In [1]:
import os
os.getcwd()844424930131970

'/home/afidegnum/projects/org'

In [21]:
# cursor = ag.execCypher("MATCH (n:node {tag: 'html'}) RETURN n")
x = 844424930131970
children = ag.execCypher("MATCH (v:node)-[R:connect]-(V2) WHERE id(v) = 844424930131969 RETURN V2")
for i in children:
    print("---")
    ch = ag.execCypher("MATCH (v:node)-[R:connect]-(V2) WHERE id(v) = %s RETURN V2", params=(i[0].id,))
    for f in ch:
        print(f)
# t = [x[0].id for x in cursor]
# print(t[0])
# cursor = ag.execCypher("MATCH (v:node)-[R:connect]-(V2) WHERE id(v) = 844424930131969 RETURN v, V2")
# for c in cursor:
# print(cur[0].id)
# for row in cursor:
#     vertex = row[0]
#     # print(row)
#     # print(vertex.id, vertex.label, vertex["tag"])
#     print(vertex["tag"])

---
({label:node, id:844424930131969, properties:{tag: html, attrib: {\"lang\": \"en\"}, }}::VERTEX,)
({label:node, id:844424930131971, properties:{tag: meta, attrib: {\"charset\": \"UTF-8\"}, }}::VERTEX,)
({label:node, id:844424930131972, properties:{tag: title, attrib: null, }}::VERTEX,)
---
({label:node, id:844424930131969, properties:{tag: html, attrib: {\"lang\": \"en\"}, }}::VERTEX,)
({label:node, id:844424930131974, properties:{tag: ul, attrib: {\"class\": \"menu\"}, }}::VERTEX,)
({label:node, id:844424930131978, properties:{tag: div, attrib: {\"id\": \"idone\", \"class\": \"classone\"}, }}::VERTEX,)
({label:node, id:844424930131987, properties:{tag: div, attrib: {\"id\": \"second\", \"class\": \"below\"}, }}::VERTEX,)


In [29]:
ag.close()

In [5]:
# Clear test data
age.deleteGraph(ag.connection, GRAPH_NAME)
# connection close
ag.commit()
